In [ ]:
# -*- coding:utf-8 -*-
import pymongo
import requests
import multiprocessing
import re
import datetime
import random
import time  
import os  
import sched  

INTERVAL = 1800
# 初始化sched模块的scheduler类  
# 第一个参数是一个可以返回时间戳的函数，第二个参数可以在定时未到达之前阻塞。  
schedule = sched.scheduler(time.time, time.sleep)

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice', 'locachoice')
    return conn

def get_mongo_crawl_coll():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn['crawl_gzjt_gov'] 

def req_traffic_data():
    url_base = 'http://219.136.133.162/gztraffic/GetData.ashx?__r='    
    timeStamp = str(int(time.time()))
    url = url_base + timeStamp
    print(url)
    response  = requests.get(url)
    if response.status_code is not 200: return 404
    jdoc = response.json()
    roads = jdoc.get('RoadData')
    coll = get_mongo_crawl_coll() 
    for road in roads:
        coll.insert_one(road)
    return 200

# 被周期性调度触发的函数  
def execute_command(cmd, inc):  
    ''' 
    执行制定请求
    '''  
    print(cmd, int(time.time()))
    req_traffic_data()
    schedule.enter(inc, 0, execute_command, (cmd, inc))  
  
  
def main(cmd, inc=1800):  
    # enter四个参数分别为：间隔事件、优先级（用于同时间到达的两个事件同时执行时定序）、被调用触发的函数，  
    # 给该触发函数的参数（tuple形式）  
    schedule.enter(0, 0, execute_command, (cmd, inc))  
    schedule.run()  
  
  
# 每60秒查看下网络连接情况  
if __name__ == '__main__':  
    main('reques_traffic', INTERVAL) 
    